In [ ]:
# import dependencies
import pandas as pd
import mysql.connector as mc

# Get db credentials
from configTest import mysql_host, mysql_u, mysql_pw #, vgc_host, vgc_u, vgc_pw, svr, db, sql_u, sql_pw, smtp_host, e_user, e_pw, port

In [ ]:
# connect to claim_qb_payments db
cnx = mc.connect(user=mysql_u, password=mysql_pw,
                host=mysql_host,
                database='claim_qb_payments')
cursor = cnx.cursor()

In [ ]:
# sql query for GAP claims that are RTBP
sql_file = '''

            '''

In [ ]:
# execute sql
cursor.execute(sql_file)

In [ ]:
# save query results as DF
df = pd.DataFrame(cursor.fetchall())

In [ ]:
# add column names
df_cols = []

df.columns = df_cols

In [ ]:
# close mysql connection
cursor.close()
cnx.close()